In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle

In [3]:
## Load Dataset

data = pd.read_csv("Churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
## Preprocess the data
### Drop irrelevent features

data = data.drop(['RowNumber', 'CustomerId', 'Surname'],axis = 1)    
##axis = 1 means coulumnwise
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [5]:
## Encode Categorical Variable
label_encoder_gender = LabelEncoder()
data['Gender']= label_encoder_gender.fit_transform(data['Gender'])

In [6]:
data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64,0
9996,516,France,1,35,10,57369.61,1,1,1,101699.77,0
9997,709,France,0,36,7,0.00,1,0,1,42085.58,1
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52,1


In [7]:
## onehotEncoder ['Geography]
from sklearn.preprocessing import OneHotEncoder
onehot = OneHotEncoder()
geo_encoder = onehot.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [8]:
onehot.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [9]:
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(), columns = onehot.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [11]:
## Combine all column with OHE data
data= pd.concat([data.drop('Geography', axis=1), geo_encoded_df],axis=1)

In [12]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
## Save the encoders and scaler
with open('label_encoder_gender.pkl','wb') as file:
    pickle.dump(label_encoder_gender, file)

with open('onehot.pkl','wb') as file:
    pickle.dump(onehot, file)

In [14]:
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [15]:
## Divide the dataset into independent and dependent dataset

X= data.drop('Exited', axis=1)
y= data['Exited']

## Split the data in training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X,y,train_size=0.2, random_state = 42)

##Scale these features
#We dop this to reduce the value to some small value to increase computation speed.
scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_test= scaler.transform(X_test)


In [16]:
X_train

array([[-0.38381126, -1.12244688,  2.60786056, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [-0.73723819, -1.12244688,  0.12136034, ..., -0.99600797,
         1.74133801, -0.58350885],
       [ 0.32304261,  0.89091075, -0.54808203, ...,  1.00400803,
        -0.57427105, -0.58350885],
       ...,
       [ 0.85318301, -1.12244688, -0.06990891, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [ 0.14632915,  0.89091075,  0.40826421, ...,  1.00400803,
        -0.57427105, -0.58350885],
       [ 0.45817644,  0.89091075,  1.1733412 , ..., -0.99600797,
         1.74133801, -0.58350885]])

In [17]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

ANN Implementation

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [19]:
X_train.shape[1]

12

In [20]:
### Build ANN Model
model = Sequential([
    Dense(64, activation= 'relu', input_shape=(X_train.shape[1],)),
    Dense(32, activation= 'relu'),
    Dense(1, activation= 'sigmoid')      #output layer

])


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
import tensorflow
opt= tensorflow.keras.optimizers.Adam(learning_rate=0.01)
loss=tensorflow.keras.losses.BinaryCrossentropy

In [23]:
## Compile the model
model.compile(optimizer=opt, loss="binary_crossentropy", metrics = ['accuracy'])

In [24]:
## Set up the Tensorboard
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
log_dir = "log/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq = 1)

In [25]:
## Setup Early stopping
early_stop_callback = EarlyStopping(monitor = 'val_loss', patience=10, restore_best_weights=True)


In [26]:
### Training the Model
history = model.fit(
    X_train, y_train, validation_data = (X_test, y_test), epochs =100,
    callbacks =[tensorflow_callback, early_stop_callback]
)

Epoch 1/100


63/63 [==============================] - 2s 11ms/step - loss: 0.4510 - accuracy: 0.8130 - val_loss: 0.4052 - val_accuracy: 0.8321
Epoch 2/100
63/63 [==============================] - 0s 7ms/step - loss: 0.3876 - accuracy: 0.8340 - val_loss: 0.3788 - val_accuracy: 0.8439
Epoch 3/100
63/63 [==============================] - 0s 7ms/step - loss: 0.3698 - accuracy: 0.8485 - val_loss: 0.3705 - val_accuracy: 0.8419
Epoch 4/100
63/63 [==============================] - 0s 8ms/step - loss: 0.3558 - accuracy: 0.8510 - val_loss: 0.3655 - val_accuracy: 0.8500
Epoch 5/100
63/63 [==============================] - 0s 7ms/step - loss: 0.3441 - accuracy: 0.8590 - val_loss: 0.3677 - val_accuracy: 0.8520
Epoch 6/100
63/63 [==============================] - 0s 8ms/step - loss: 0.3406 - accuracy: 0.8580 - val_loss: 0.3660 - val_accuracy: 0.8526
Epoch 7/100
63/63 [==============================] - 0s 6ms/step - loss: 0.3281 - accuracy: 0.8645 - val_loss: 0.3808 - val_accuracy: 0.8510
Epoch 8/10

In [27]:
model.save('model.h5')

c:\Users\siddh\Desktop\NEW_Course\Project\ANN_Comparision\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [28]:
## Load tensorBoard extension

%load_ext tensorboard

In [29]:

%tensorboard --logdir log/fit20250616-234834

Reusing TensorBoard on port 6007 (pid 8620), started 14:42:46 ago. (Use '!kill 8620' to kill it.)